<a href="https://colab.research.google.com/github/joanby/tensorflow2/blob/master/Collab%208%20-%20Reinforcement%20Learning%20para%20problemas%20de%20Stock%20Market%20Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Paso 1: Instalar las dependencias y configurar el entorno

In [ ]:
#!pip install tensorflow-gpu==2.0.0.alpha0
%tensorflow_version 2.x

In [1]:
!pip install pandas-datareader

## Paso 2: Importar las dependencias del proyecto

In [2]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

In [3]:
tf.__version__

'2.4.1'

## Paso 3: Construir la red neuronal de la AI del Trader 

In [4]:
class AI_Trader():
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Manten, Compra, Vende
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name
    
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    
    self.model = self.model_builder()
    
  def model_builder(self):
    
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))
    
    return model
  
  def trade(self, state):
    
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  
  def batch_train(self, batch_size):
    
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

## Paso 4: Pre procesado del dataset

### Definir las funciones adicionales

#### Sigmoide

In [5]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

#### Función de formato de precios

In [6]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

#### Carga del dataset

In [7]:
def dataset_loader(stock_name):
  
  #Complete the dataset loader function
  dataset = data_reader.DataReader(stock_name, data_source="yahoo")
  
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  
  close = dataset['Close']

  return close

### State creator

In [8]:
def state_creator(data, timestep, window_size):
  
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])

### Cargar una divisa de mercado

In [9]:
stock_name = "AAPL"
data = dataset_loader(stock_name)

In [10]:
data.head()

Date
2016-04-08    27.165001
2016-04-11    27.254999
2016-04-12    27.610001
2016-04-13    28.010000
2016-04-14    28.025000
Name: Close, dtype: float64

## Paso 5: Entrenar la AI Trader

### Configurar los hyper parámetros

In [12]:
window_size = 10
episodes = 1000

batch_size = 32
data_samples = len(data) - 1

### Definir el modelo del AI Trader

windows_size va a ser 10 porque va a estar viendo 10 dias en el pasado

In [13]:
trader = AI_Trader(window_size)

In [14]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


### Bucle de entrenamiento

In [ ]:
for episode in range(1, episodes + 1):
  
  print("Episodio: {}/{}".format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    # ahora quiero predecir el dia siguiente t +1 , siempre existe el dia siguiente
    next_state = state_creator(data, t+1, window_size + 1)
    # ahora necesitamos definir la recompensa
    reward = 0
    
    if action == 1: #Compra
      # marco que la gente compro y lo añado
      trader.inventory.append(data[t])
      # imprimo que compro
      print("AI Trader compró: ", stocks_price_format(data[t]))
      
    # si la accion es de venta
    # solo puedo vende si el inventario es positivo
    elif action == 2 and len(trader.inventory) > 0: #Vende
      # saca el ultimo elemento que tenemos, la ultima accion, es independiente de a que valor se compro
      buy_price = trader.inventory.pop(0)
      
      # recompensa el maximo del dia dela compra menos el precio de compra de la accion, solo me interesa resultados positivos no los negativos
      # si da negativo pongo 0 porque no me interesa las reward que no me dan ganacia
      reward = max(data[t] - buy_price, 0)
      # acumulamos el beneficio con  valor de venta actual de la accion - valor de compra de la accion
      total_profit += data[t] - buy_price
      # imprimimos cuanto vendio el trader
      print("AI Trader vendió: ", stocks_price_format(data[t]), " Beneficio: " + stocks_price_format(data[t] - buy_price) )
    
    # nos determina si estamos al final del dataset, si es asi finalizamos el episodio
    if t == data_samples - 1:
      done = True
    else:
      done = False
    
    # actualizamos los pesos de la red neuronal profunda
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    # imprimo el beneficio total del episodio
    if done:
      print("########################")
      print("BENEFICIO TOTAL: {}".format(total_profit))
      print("########################")
    # fase del final, cada vez que superamos el batch size hay que actualizar los pesos de la red
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
  # cada 10 episodios guardo el modelo con el nombre del archivo con el final del nro de episodio
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))
    

  0%|          | 0/1257 [00:00<?, ?it/s]

Episodio: 1/1000
AI Trader compró:  $ 27.254999
AI Trader vendió:  $ 28.010000  Beneficio: $ 0.755001
AI Trader compró:  $ 28.025000
AI Trader compró:  $ 26.870001
AI Trader compró:  $ 26.782499
AI Trader compró:  $ 26.420000
AI Trader vendió:  $ 26.087500  Beneficio: - $ 1.937500
AI Trader vendió:  $ 23.707500  Beneficio: - $ 3.162500
AI Trader compró:  $ 23.410000
AI Trader vendió:  $ 23.795000  Beneficio: - $ 2.987499
AI Trader vendió:  $ 23.309999  Beneficio: - $ 3.110001
AI Trader vendió:  $ 23.180000  Beneficio: - $ 0.230000
AI Trader compró:  $ 23.355000
AI Trader compró:  $ 22.584999
AI Trader compró:  $ 22.629999
AI Trader vendió:  $ 23.805000  Beneficio: $ 0.450001
AI Trader compró:  $ 24.475000


  3%|▎         | 33/1257 [00:05<03:41,  5.52it/s]

AI Trader vendió:  $ 24.905001  Beneficio: $ 2.320002


  3%|▎         | 35/1257 [00:11<31:14,  1.53s/it]

...

 43%|████▎     | 536/1257 [24:49<35:33,  2.96s/it]

AI Trader vendió:  $ 47.037498  Beneficio: $ 0.524998


 43%|████▎     | 537/1257 [24:52<35:06,  2.93s/it]

AI Trader vendió:  $ 47.145000  Beneficio: $ 0.305000


 44%|████▎     | 547/1257 [25:22<35:02,  2.96s/it]